## 성진님 feature :
'temp', 'pred', 'wind', 'hum', 'day', 'month', 'week', 'sin_time', 'cos_time', 'THI', 'feels_temp', 'CDH', 'hour_mean', 'heat_index'(구간나눔), 'heat_wave(뺴도될 듯)', 'day_hour_mean', 'hour_std', 'last_week_power'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

DATA_PATH = "/content/drive/MyDrive/데이콘 캐글 컴페티션/DACON_23.07_PowerConsumption/"
SEED = 42

In [3]:
import matplotlib.pyplot as plt

In [4]:
train_ft = pd.read_csv(f"{DATA_PATH}train.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test.csv")
building_info = pd.read_csv(f"{DATA_PATH}building_info.csv")

In [ ]:
# train_df = train_df[(train_df['건물번호'] >= 69) & (train_df['건물번호'] <= 100)]
# test_df = test_df[(test_df['건물번호'] >= 69) & (test_df['건물번호'] <= 100)]
# building_info = building_info[(building_info['건물번호'] >= 69) & (building_info['건물번호'] <= 100)]

In [5]:
train_ft

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40
...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,20220824 19,23.1,NaN,0.9,86.0,0.5,NaN,881.04
203996,100_20220824 20,100,20220824 20,22.4,NaN,1.3,86.0,0.0,NaN,798.96
203997,100_20220824 21,100,20220824 21,21.3,NaN,1.0,92.0,NaN,NaN,825.12
203998,100_20220824 22,100,20220824 22,21.0,NaN,0.3,94.0,NaN,NaN,640.08


In [6]:
train_ft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204000 entries, 0 to 203999
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   num_date_time  204000 non-null  object 
 1   건물번호           204000 non-null  int64  
 2   일시             204000 non-null  object 
 3   기온(C)          204000 non-null  float64
 4   강수량(mm)        43931 non-null   float64
 5   풍속(m/s)        203981 non-null  float64
 6   습도(%)          203991 non-null  float64
 7   일조(hr)         128818 non-null  float64
 8   일사(MJ/m2)      116087 non-null  float64
 9   전력소비량(kWh)     204000 non-null  float64
dtypes: float64(7), int64(1), object(2)
memory usage: 15.6+ MB


In [ ]:
test_ft

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%)
0,1_20220825 00,1,20220825 00,23.5,0.0,2.2,72
1,1_20220825 01,1,20220825 01,23.0,0.0,0.9,72
2,1_20220825 02,1,20220825 02,22.7,0.0,1.5,75
3,1_20220825 03,1,20220825 03,22.1,0.0,1.3,78
4,1_20220825 04,1,20220825 04,21.8,0.0,1.0,77
...,...,...,...,...,...,...,...
16795,100_20220831 19,100,20220831 19,22.5,0.0,0.9,84
16796,100_20220831 20,100,20220831 20,20.7,0.0,0.4,95
16797,100_20220831 21,100,20220831 21,20.2,0.0,0.4,98
16798,100_20220831 22,100,20220831 22,20.1,0.0,1.1,97


In [ ]:
test_ft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16800 entries, 0 to 16799
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   num_date_time  16800 non-null  object 
 1   건물번호           16800 non-null  int64  
 2   일시             16800 non-null  object 
 3   기온(C)          16800 non-null  float64
 4   강수량(mm)        16800 non-null  float64
 5   풍속(m/s)        16800 non-null  float64
 6   습도(%)          16800 non-null  int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 918.9+ KB


In [ ]:
building_info

,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,110634.00,39570.00,-,-,-
1,2,건물기타,122233.47,99000.00,-,-,-
2,3,건물기타,171243.00,113950.00,40,-,-
3,4,건물기타,74312.98,34419.62,60,-,-
4,5,건물기타,205884.00,150000.00,-,2557,1000
...,...,...,...,...,...,...,...
95,96,호텔및리조트,93314.00,60500.00,-,-,-
96,97,호텔및리조트,55144.67,25880.00,-,-,-
97,98,호텔및리조트,53578.62,17373.75,-,-,-
98,99,호텔및리조트,53499.00,40636.00,-,-,-


In [7]:
building_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   건물번호          100 non-null    int64  
 1   건물유형          100 non-null    object 
 2   연면적(m2)       100 non-null    float64
 3   냉방면적(m2)      100 non-null    float64
 4   태양광용량(kW)     100 non-null    object 
 5   ESS저장용량(kWh)  100 non-null    object 
 6   PCS용량(kW)     100 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 5.6+ KB


In [8]:
building_info['태양광용량(kW)'] = building_info['태양광용량(kW)'].replace('-', 0).astype(float)
building_info['ESS저장용량(kWh)'] = building_info['ESS저장용량(kWh)'].replace('-', 0).astype(float)
building_info['PCS용량(kW)'] = building_info['PCS용량(kW)'].replace('-', 0).astype(float)

In [ ]:
building_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   건물번호          100 non-null    int64  
 1   건물유형          100 non-null    object 
 2   연면적(m2)       100 non-null    float64
 3   냉방면적(m2)      100 non-null    float64
 4   태양광용량(kW)     100 non-null    float64
 5   ESS저장용량(kWh)  100 non-null    float64
 6   PCS용량(kW)     100 non-null    float64
dtypes: float64(5), int64(1), object(1)
memory usage: 5.6+ KB


In [9]:
train_ft = pd.merge(train_ft, building_info, on='건물번호', how='left')
test_ft = pd.merge(test_ft, building_info, on='건물번호', how='left')

In [10]:
# num_date_time, 태양광, ESS, PCS 제거
train_ft = train_ft.drop(columns=['num_date_time', '태양광용량(kW)','ESS저장용량(kWh)', 'PCS용량(kW)'])
test_ft = test_ft.drop(columns=['num_date_time', '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)'])

In [11]:
train_ft.columns

Index(['건물번호', '일시', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '일조(hr)',
       '일사(MJ/m2)', '전력소비량(kWh)', '건물유형', '연면적(m2)', '냉방면적(m2)'],
      dtype='object')

In [12]:
# 변수 영문명으로 변경
train_cols = ['building_num', 'date_time', 'temp', 'precip', 'wind', 'hum', 'sunshine', 'solar_rad', 'energy_consum', 'building_type', 'floor_area', 'cooling_area']
test_cols = ['building_num', 'date_time', 'temp', 'precip', 'wind', 'hum', 'building_type', 'floor_area', 'cooling_area']

train_ft.columns = train_cols
test_ft.columns = test_cols

# 전력량 맨 앞으로 이동
train_ft = train_ft[['energy_consum','building_num', 'date_time', 'temp', 'precip', 'wind', 'hum', 'sunshine', 'solar_rad', 'building_type', 'floor_area', 'cooling_area']]

In [ ]:
train_ft.head()

,energy_consum,building_num,date_time,temp,precip,wind,hum,sunshine,solar_rad,building_type,floor_area,cooling_area
0,1085.28,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,건물기타,110634.0,39570.0
1,1047.36,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,건물기타,110634.0,39570.0
2,974.88,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,건물기타,110634.0,39570.0
3,953.76,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,건물기타,110634.0,39570.0
4,986.40,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,건물기타,110634.0,39570.0


In [ ]:
test_ft.head()

,building_num,date_time,temp,precip,wind,hum,building_type,floor_area,cooling_area
0,1,20220825 00,23.5,0.0,2.2,72,건물기타,110634.0,39570.0
1,1,20220825 01,23.0,0.0,0.9,72,건물기타,110634.0,39570.0
2,1,20220825 02,22.7,0.0,1.5,75,건물기타,110634.0,39570.0
3,1,20220825 03,22.1,0.0,1.3,78,건물기타,110634.0,39570.0
4,1,20220825 04,21.8,0.0,1.0,77,건물기타,110634.0,39570.0


---

#### train data

In [13]:
#시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
date = pd.to_datetime(train_ft['date_time'])
train_ft['time'] = date.dt.hour
train_ft['day'] = date.dt.weekday
train_ft['month'] = date.dt.month
train_ft['week'] = date.dt.isocalendar().week
train_ft['day_of_year'] = date.dt.dayofyear

train_ft['sin_time'] = np.sin(2*np.pi*train_ft['time']/24)
train_ft['cos_time'] = np.cos(2*np.pi*train_ft['time']/24)

In [ ]:
# time 칼럼 제거
# train_df = train_df.drop(columns=['time'])

In [14]:
# 강수랑, 일조, 일사, 풍속 결측치 0.0으로 채우기
train_ft['precip'] = train_ft['precip'].fillna(0.0)
train_ft['sunshine'] = train_ft['sunshine'].fillna(0.0)
train_ft['solar_rad'] = train_ft['solar_rad'].fillna(0.0)
train_ft['wind'] = train_ft['wind'].fillna(0.0)

In [ ]:
train_ft['THI'] = 1.8*train_ft['temp'] - 0.55*(1-train_ft['hum']/100)*(1.8*train_ft['temp']-26)+32
train_ft

,energy_consum,building_num,date_time,temp,precip,wind,hum,sunshine,solar_rad,building_type,floor_area,cooling_area,time,day,month,week,day_of_year,sin_time,cos_time,THI
0,1085.28,1,20220601 00,18.6,0.0,0.9,42.0,0.0,0.0,건물기타,110634.00,39570.00,0,2,6,22,152,0.000000,1.000000,63.09388
1,1047.36,1,20220601 01,18.0,0.0,1.1,45.0,0.0,0.0,건물기타,110634.00,39570.00,1,2,6,22,152,0.258819,0.965926,62.46400
2,974.88,1,20220601 02,17.7,0.0,1.5,45.0,0.0,0.0,건물기타,110634.00,39570.00,2,2,6,22,152,0.500000,0.866025,62.08735
3,953.76,1,20220601 03,16.7,0.0,1.4,48.0,0.0,0.0,건물기타,110634.00,39570.00,3,2,6,22,152,0.707107,0.707107,60.89884
4,986.40,1,20220601 04,18.4,0.0,2.8,43.0,0.0,0.0,건물기타,110634.00,39570.00,4,2,6,22,152,0.866025,0.500000,62.88788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,881.04,100,20220824 19,23.1,0.0,0.9,86.0,0.5,0.0,호텔및리조트,57497.84,40035.23,19,2,8,34,236,-0.965926,0.258819,72.38034
203996,798.96,100,20220824 20,22.4,0.0,1.3,86.0,0.0,0.0,호텔및리조트,57497.84,40035.23,20,2,8,34,236,-0.866025,0.500000,71.21736
203997,825.12,100,20220824 21,21.3,0.0,1.0,92.0,0.0,0.0,호텔및리조트,57497.84,40035.23,21,2,8,34,236,-0.707107,0.707107,69.79704
203998,640.08,100,20220824 22,21.0,0.0,0.3,94.0,0.0,0.0,호텔및리조트,57497.84,40035.23,22,2,8,34,236,-0.500000,0.866025,69.41060



---
#### test data


In [ ]:
date = pd.to_datetime(test_ft['date_time'])
test_ft['time'] = date.dt.hour
test_ft['day'] = date.dt.weekday
test_ft['month'] = date.dt.month
test_ft['week'] = date.dt.isocalendar().week
test_ft['day_of_year'] = date.dt.dayofyear

test_ft['sin_time'] = np.sin(2*np.pi*test_ft['time']/24)
test_ft['cos_time'] = np.cos(2*np.pi*test_ft['time']/24)

In [ ]:
# time 칼럼 제거
# test_df = test_df.drop(columns=['time'])

In [ ]:
test_ft['THI'] = 1.8*test_ft['temp'] - 0.55*(1 - test_ft['hum']/100)*(1.8*test_ft['temp'] - 26)+32
test_ft

,building_num,date_time,temp,precip,wind,hum,building_type,floor_area,cooling_area,time,day,month,week,day_of_year,sin_time,cos_time,THI
0,1,20220825 00,23.5,0.0,2.2,72,건물기타,110634.00,39570.00,0,3,8,34,237,0.000000,1.000000,71.78980
1,1,20220825 01,23.0,0.0,0.9,72,건물기타,110634.00,39570.00,1,3,8,34,237,0.258819,0.965926,71.02840
2,1,20220825 02,22.7,0.0,1.5,75,건물기타,110634.00,39570.00,2,3,8,34,237,0.500000,0.866025,70.81675
3,1,20220825 03,22.1,0.0,1.3,78,건물기타,110634.00,39570.00,3,3,8,34,237,0.707107,0.707107,70.11262
4,1,20220825 04,21.8,0.0,1.0,77,건물기타,110634.00,39570.00,4,3,8,34,237,0.866025,0.500000,69.56514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,20220831 19,22.5,0.0,0.9,84,호텔및리조트,57497.84,40035.23,19,2,8,35,243,-0.965926,0.258819,71.22400
16796,100,20220831 20,20.7,0.0,0.4,95,호텔및리조트,57497.84,40035.23,20,2,8,35,243,-0.866025,0.500000,68.95035
16797,100,20220831 21,20.2,0.0,0.4,98,호텔및리조트,57497.84,40035.23,21,2,8,35,243,-0.707107,0.707107,68.24604
16798,100,20220831 22,20.1,0.0,1.1,97,호텔및리조트,57497.84,40035.23,22,2,8,35,243,-0.500000,0.866025,68.01203


In [ ]:
# 필요 없는 칼럼 제거
train_ft = train_ft.drop(columns=['date_time', 'sunshine', 'solar_rad','building_type'])
test_ft = test_ft.drop(columns=['date_time', 'building_type'])

In [ ]:
train_ft.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204000 entries, 0 to 203999
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   energy_consum  204000 non-null  float64
 1   building_num   204000 non-null  int64  
 2   temp           204000 non-null  float64
 3   precip         204000 non-null  float64
 4   wind           204000 non-null  float64
 5   hum            203991 non-null  float64
 6   floor_area     204000 non-null  float64
 7   cooling_area   204000 non-null  float64
 8   time           204000 non-null  int64  
 9   day            204000 non-null  int64  
 10  month          204000 non-null  int64  
 11  week           204000 non-null  UInt32 
 12  day_of_year    204000 non-null  int64  
 13  sin_time       204000 non-null  float64
 14  cos_time       204000 non-null  float64
 15  THI            203991 non-null  float64
dtypes: UInt32(1), float64(10), int64(5)
memory usage: 25.9 MB


In [ ]:
test_ft.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16800 entries, 0 to 16799
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   building_num  16800 non-null  int64  
 1   temp          16800 non-null  float64
 2   precip        16800 non-null  float64
 3   wind          16800 non-null  float64
 4   hum           16800 non-null  int64  
 5   floor_area    16800 non-null  float64
 6   cooling_area  16800 non-null  float64
 7   time          16800 non-null  int64  
 8   day           16800 non-null  int64  
 9   month         16800 non-null  int64  
 10  week          16800 non-null  UInt32 
 11  day_of_year   16800 non-null  int64  
 12  sin_time      16800 non-null  float64
 13  cos_time      16800 non-null  float64
 14  THI           16800 non-null  float64
dtypes: UInt32(1), float64(8), int64(6)
memory usage: 2.0 MB


In [ ]:
train_ft['week'] = train_ft['week'].astype(int)
test_ft['week'] = test_ft['week'].astype(int)

In [ ]:
train_ft

,energy_consum,building_num,temp,precip,wind,hum,floor_area,cooling_area,time,day,month,week,day_of_year,sin_time,cos_time,THI
0,1085.28,1,18.6,0.0,0.9,42.0,110634.00,39570.00,0,2,6,22,152,0.000000,1.000000,63.09388
1,1047.36,1,18.0,0.0,1.1,45.0,110634.00,39570.00,1,2,6,22,152,0.258819,0.965926,62.46400
2,974.88,1,17.7,0.0,1.5,45.0,110634.00,39570.00,2,2,6,22,152,0.500000,0.866025,62.08735
3,953.76,1,16.7,0.0,1.4,48.0,110634.00,39570.00,3,2,6,22,152,0.707107,0.707107,60.89884
4,986.40,1,18.4,0.0,2.8,43.0,110634.00,39570.00,4,2,6,22,152,0.866025,0.500000,62.88788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,881.04,100,23.1,0.0,0.9,86.0,57497.84,40035.23,19,2,8,34,236,-0.965926,0.258819,72.38034
203996,798.96,100,22.4,0.0,1.3,86.0,57497.84,40035.23,20,2,8,34,236,-0.866025,0.500000,71.21736
203997,825.12,100,21.3,0.0,1.0,92.0,57497.84,40035.23,21,2,8,34,236,-0.707107,0.707107,69.79704
203998,640.08,100,21.0,0.0,0.3,94.0,57497.84,40035.23,22,2,8,34,236,-0.500000,0.866025,69.41060


In [ ]:
# 냉방면적 비율 추가

train_ft['cooling_ratio'] = train_ft['cooling_area'] / train_ft['floor_area']
test_ft['cooling_ratio'] = test_ft['cooling_area'] / test_ft['floor_area']

In [ ]:
# 체감온도 변수 추가
def wet_temp(temp, hum):
    tw = temp*np.arctan(0.151977*(hum + 8.313659)*0.5) + np.arctan(temp + hum) - np.arctan(hum - 1.67633) + 0.00391838*1.5*np.arctan(0.023101*hum) - 4.686035
    return tw

def feels_temp(temp, hum):
    tw = wet_temp(temp, hum)
    ft = -0.2442 + 0.55399*tw + 0.45535*temp - 0.0022*(tw**2) + 0.00278*tw*temp + 3.0
    return ft

In [ ]:
train_ft['feels_temp'] = feels_temp(train_ft['temp'], train_ft['hum'])

test_ft['feels_temp'] = feels_temp(test_ft['temp'], test_ft['hum'])

In [ ]:
# cdh(cooling degree hour) 변수 추가
def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

In [ ]:
cdhs = np.array([])
for num in range(1,101):
    temp = train_ft[train_ft['building_num'] == num]
    cdh = CDH(temp['temp'].values)
    cdhs = np.concatenate([cdhs, cdh])
train_ft['CDH'] = cdhs

cdhs = np.array([])
for num in range(1,101):
    temp = test_ft[test_ft['building_num'] == num]
    cdh = CDH(temp['temp'].values)
    cdhs = np.concatenate([cdhs, cdh])
test_ft['CDH'] = cdhs

In [ ]:
# 6월 1일 삭제 --> day_of_year == 152
mask_train = train_ft['day_of_year'] != 152
train_ft = train_ft[mask_train]

In [ ]:
# 시간별 전력량 평균 추가
power_hour_mean = train_ft.groupby(['building_num', 'time'])['energy_consum'].mean().reset_index()
train_ft['hour_mean'] = train_ft.apply(lambda x :power_hour_mean.loc[(power_hour_mean.building_num == x['building_num']) & (power_hour_mean.time == x['time']), 'energy_consum'].values[0], axis=1)
test_ft['hour_mean'] = test_ft.apply(lambda x :power_hour_mean.loc[(power_hour_mean.building_num == x['building_num']) & (power_hour_mean.time == x['time']), 'energy_consum'].values[0], axis=1)

<ipython-input-38-bce7803fff3d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ft['hour_mean'] = train_ft.apply(lambda x :power_hour_mean.loc[(power_hour_mean.building_num == x['building_num']) & (power_hour_mean.time == x['time']), 'energy_consum'].values[0], axis=1)


- 열지수 구간

In [ ]:
# 열지수 단계별로 구분
T = train_ft['temp']*9/5 + 32 # 화씨로 전환
RH = train_ft['hum']
train_ft['heat_index'] = -42.379 + 2.04901523*T + 10.14333127*RH - .22475541*T*RH - .00683783*T*T - .05481717*RH*RH + .00122874*T*T*RH + .00085282*T*RH*RH - .00000199*T*T*RH*RH
train_ft['heat_index'] = (train_ft['heat_index']-32) * 5/9 # 섭씨로 전환
train_ft.loc[train_ft['heat_index']<32, 'heat_index'] = 0
train_ft.loc[(train_ft['heat_index']>=32) & (train_ft['heat_index']<41), 'heat_index'] = 1
train_ft.loc[(train_ft['heat_index']>=41) & (train_ft['heat_index']<54), 'heat_index'] = 2
train_ft.loc[(train_ft['heat_index']>=54) & (train_ft['heat_index']<66), 'heat_index'] = 3
train_ft.loc[train_ft['heat_index']>=66, 'heat_index'] = 4

<ipython-input-39-5ed69ac64263>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ft['heat_index'] = -42.379 + 2.04901523*T + 10.14333127*RH - .22475541*T*RH - .00683783*T*T - .05481717*RH*RH + .00122874*T*T*RH + .00085282*T*RH*RH - .00000199*T*T*RH*RH
<ipython-input-39-5ed69ac64263>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ft['heat_index'] = (train_ft['heat_index']-32) * 5/9 # 섭씨로 전환


In [ ]:
T = test_ft['temp']*9/5 + 32 # 화씨로 전환
RH = test_ft['hum']
test_ft['heat_index'] = -42.379 + 2.04901523*T + 10.14333127*RH - .22475541*T*RH - .00683783*T*T - .05481717*RH*RH + .00122874*T*T*RH + .00085282*T*RH*RH - .00000199*T*T*RH*RH
test_ft['heat_index'] = (test_ft['heat_index']-32) * 5/9 # 섭씨로 전환
test_ft.loc[test_ft['heat_index']<32, 'heat_index'] = 0
test_ft.loc[(test_ft['heat_index']>=32) & (test_ft['heat_index']<41), 'heat_index'] = 1
test_ft.loc[(test_ft['heat_index']>=41) & (test_ft['heat_index']<54), 'heat_index'] = 2
test_ft.loc[(test_ft['heat_index']>=54) & (test_ft['heat_index']<66), 'heat_index'] = 3
test_ft.loc[test_ft['heat_index']>=66, 'heat_index'] = 4

- 폭염 여부

In [ ]:
# # 폭염 여부 추가
# train_ft['heat_wave'] = 0
# train_ft.loc[train_ft['temp'] >= 33, 'heat_wave'] = 1

# test_ft['heat_wave'] = 0
# test_ft.loc[test_ft['temp'] >= 33, 'heat_wave'] = 1

- day hour mean, day std

In [ ]:
power_mean = pd.pivot_table(train_ft, values = 'energy_consum', index = ['building_num', 'time', 'day'], aggfunc = np.mean).reset_index()

train_ft['day_hour_mean'] = train_ft.apply(lambda x : power_mean.loc[(power_mean.building_num == x['building_num']) & (power_mean.time == x['time']) & (power_mean.day == x['day']) ,'energy_consum'].values[0], axis = 1)
test_ft['day_hour_mean'] = test_ft.apply(lambda x : power_mean.loc[(power_mean.building_num == x['building_num']) & (power_mean.time == x['time']) & (power_mean.day == x['day']) ,'energy_consum'].values[0], axis = 1)

<ipython-input-42-ac098408ee07>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ft['day_hour_mean'] = train_ft.apply(lambda x : power_mean.loc[(power_mean.building_num == x['building_num']) & (power_mean.time == x['time']) & (power_mean.day == x['day']) ,'energy_consum'].values[0], axis = 1)


In [ ]:
power_hour_std = pd.pivot_table(train_ft, values = 'energy_consum', index = ['building_num', 'time'], aggfunc = np.std).reset_index()

train_ft['hour_std'] = train_ft.apply(lambda x : power_hour_std.loc[(power_hour_std.building_num == x['building_num']) & (power_hour_std.time == x['time']) ,'energy_consum'].values[0], axis = 1)
test_ft['hour_std'] = test_ft.apply(lambda x : power_hour_std.loc[(power_hour_std.building_num == x['building_num']) & (power_hour_std.time == x['time']) ,'energy_consum'].values[0], axis = 1)

<ipython-input-44-98d2babb34c9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ft['hour_std'] = train_ft.apply(lambda x : power_hour_std.loc[(power_hour_std.building_num == x['building_num']) & (power_hour_std.time == x['time']) ,'energy_consum'].values[0], axis = 1)


In [ ]:
train_ft.columns

Index(['energy_consum', 'building_num', 'temp', 'precip', 'wind', 'hum',
       'floor_area', 'cooling_area', 'time', 'day', 'month', 'week',
       'day_of_year', 'sin_time', 'cos_time', 'THI', 'cooling_ratio',
       'feels_temp', 'CDH', 'hour_mean', 'heat_index', 'day_hour_mean',
       'hour_std'],
      dtype='object')

In [ ]:
test_ft.head(10)

,building_num,temp,precip,wind,hum,floor_area,cooling_area,time,day,month,...,sin_time,cos_time,THI,cooling_ratio,feels_temp,CDH,hour_mean,heat_index,day_hour_mean,hour_std
0,1,23.5,0.0,2.2,72,110634.0,39570.0,0,3,8,...,0.000000,1.000000e+00,71.78980,0.357666,29.281051,-2.5,1713.711429,0.0,1627.80,444.306331
1,1,23.0,0.0,0.9,72,110634.0,39570.0,1,3,8,...,0.258819,9.659258e-01,71.02840,0.357666,28.665671,-5.5,1629.468571,0.0,1550.08,437.718795
2,1,22.7,0.0,1.5,75,110634.0,39570.0,2,3,8,...,0.500000,8.660254e-01,70.81675,0.357666,28.361109,-8.8,1513.305714,0.0,1431.12,410.362488
3,1,22.1,0.0,1.3,78,110634.0,39570.0,3,3,8,...,0.707107,7.071068e-01,70.11262,0.357666,27.679709,-12.7,1443.122857,0.0,1372.20,389.916262
4,1,21.8,0.0,1.0,77,110634.0,39570.0,4,3,8,...,0.866025,5.000000e-01,69.56514,0.357666,27.289743,-16.9,1452.808571,0.0,1381.72,379.995696
5,1,21.6,0.0,1.6,81,110634.0,39570.0,5,3,8,...,0.965926,2.588190e-01,69.53404,0.357666,27.115329,-21.3,1585.905714,0.0,1526.84,393.582746
6,1,21.5,0.0,2.3,84,110634.0,39570.0,6,3,8,...,1.000000,6.123234e-17,69.58240,0.357666,27.041719,-25.8,1847.397143,0.0,1866.40,401.657375
7,1,21.7,0.0,1.4,83,110634.0,39570.0,7,3,8,...,0.965926,-2.588190e-01,69.83889,0.357666,27.273401,-30.1,2214.705714,0.0,2363.96,457.177800
8,1,22.3,0.0,2.2,82,110634.0,39570.0,8,3,8,...,0.866025,-5.000000e-01,70.74014,0.357666,28.000120,-33.8,2667.514286,0.0,2829.04,455.523997
9,1,22.7,0.0,2.4,78,110634.0,39570.0,9,3,8,...,0.707107,-7.071068e-01,71.06194,0.357666,28.421422,-37.1,3101.711429,0.0,3228.64,485.613075


In [ ]:
# time 삭제, 면적 삭제
train_ft = train_ft.drop(columns=['time','floor_area', 'cooling_area', 'cooling_ratio'])
test_ft = test_ft.drop(columns=['time','floor_area', 'cooling_area', 'cooling_ratio'])

In [ ]:
train_ft.columns

Index(['energy_consum', 'building_num', 'temp', 'precip', 'wind', 'hum', 'day',
       'month', 'week', 'day_of_year', 'sin_time', 'cos_time', 'THI',
       'feels_temp', 'CDH', 'hour_mean', 'heat_index', 'day_hour_mean',
       'hour_std'],
      dtype='object')

In [ ]:
train_ft.to_csv(f'{DATA_PATH}train_feature8.csv', index=False)
test_ft.to_csv(f'{DATA_PATH}test_feature8.csv', index=False)